# Informações sobre a base de dados
* Nome do dataset: adult.csv

* Descrição: Descrição: O nome da base de dados é adult.csv onde será realizada a classificação se determinado individuo recebe <=50K ou >50K, ou seja será realizada a Avaliação de Rendas

* Link fonte de dados: https://www.kaggle.com/uciml/adult-census-income?select=adult.csv

# Dicionario de dados com o nome de cada campo e os tipos dos dados
* age: continuous.
* workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
* fnlwgt: continuous.
* education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* education-num: continuous.
* marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
* occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
* relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
* race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
* sex: Female, Male.
* capital-gain: continuous.
* capital-loss: continuous.
* hours-per-week: continuous.
* native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

# Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Biblioteca para treino, metricas 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc

# Biblioteca para validação dos algoritmos
from sklearn.model_selection import cross_val_score

# Algoritmos
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

* Criadas as classes TratamentoDados e ProcessamentoDados usadas no modulo A as mesmas serão importadas abaixo

In [2]:
from lib.processamento_dados import ProcessamentoDados as pr
from lib.tratamento_dados import TratamentoDados as tr

* Criada a classe PreparacaoDados usadas no modulo B a mesma será importada abaixo:

In [3]:
from lib.preparacao_dados import PreparacaoDados as prep

### Ignora Warnings do python

In [4]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

## Importando as bases de dados

* Na modulo A do projeto optei por utilizar uma função para renomear as colunas do dataset a fim de mostrar os valores originais na modulo B resolvi utilizar o novo nome das colunas ao importar os dados do arquivo csv

In [5]:
# cabeçalho com o nome das colunas do dataset
header_columns = ['Idade', 'Classe Trabalho', 'Peso Final', 'Educacao', 'Num Educacao', 'Estado Civil', 
                  'Ocupacao','Relacao', 'Raca', 'Sexo', 'Ganho Capital', 'Perda Capital', 'Horas Semana', 
                  'Pais Nativo', 'Renda']

In [6]:
renda = pd.read_csv("base/adult.csv", names=header_columns, skiprows=1)

## Exbindo os dados do dataset 

In [7]:
renda.head()

,Idade,Classe Trabalho,Peso Final,Educacao,Num Educacao,Estado Civil,Ocupacao,Relacao,Raca,Sexo,Ganho Capital,Perda Capital,Horas Semana,Pais Nativo,Renda
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


# Processamento e Tratamento dos dados

## Substituindo os espaços em branco por Nan

In [8]:
renda = tr.tratamento_espaco_branco(renda)

## Tratamento de dados ausentes

### Tratamento de dados ausentes na coluna Classe Trabalho

In [9]:
renda = tr.tratamento_dados_ausentes(renda, 'Classe Trabalho')

### Tratamento de dados ausentes na coluna Coluna Ocupacao

In [10]:
renda = tr.tratamento_dados_ausentes(renda, 'Ocupacao')

### Tratamento de dados ausentes na coluna Pais Nativo

In [11]:
renda = tr.tratamento_dados_ausentes(renda, 'Pais Nativo')

## Verificando se ainda temos dados ausentes
* Podemos perceber já não temos mais dados ausentes no dataset

In [12]:
renda.isnull().sum()

Idade              0
Classe Trabalho    0
Peso Final         0
Educacao           0
Num Educacao       0
Estado Civil       0
Ocupacao           0
Relacao            0
Raca               0
Sexo               0
Ganho Capital      0
Perda Capital      0
Horas Semana       0
Pais Nativo        0
Renda              0
dtype: int64

# Tratamentos de Outiliers

### Dimensão do dataset antes da remoção dos outliers

### OBS: Será feita primeiro a analise exploratória e grafica dos dados para posteriormente aplicar a limpeza dos outliers

In [13]:
renda2 = renda.copy()

In [14]:
renda2.shape

(32561, 15)

### Remove outlier Coluna Idade

In [15]:
renda2 = pr.remover_outlier_metodo_estatistico(renda2,'Idade')

### Remove outlier Coluna Horas Semana

In [16]:
renda2 = pr.remover_outlier_metodo_estatistico(renda2,'Horas Semana')

### Remove outlier Ganho Capital

In [17]:
renda2 = pr.remover_outlier_metodo_estatistico(renda2,'Ganho Capital')

### Dimensão do dataset após da remoção dos outliers¶

In [18]:
renda2.shape

(28909, 15)

# Criação dos atributos (X, y)

### Criando uma copia do dataframe

In [19]:
renda3 = renda2.copy()

### Preparando a lista de colunas auxiliares para a criação da variaveis X e y

In [20]:
# Lista de colunas para a criação do data frame
lista_colunas_aux = ['Idade', 'Ganho Capital', 'Perda Capital', 'Relacao', 'Raca','Sexo', 'Renda']

# Lista de colunas para transforma variaveis categorica em numérica
lista_colunas_dummy = ['Relacao', 'Raca','Sexo']

# Lista de colunas para deixar na mesma escala
lista_colunas_scale = ['Idade', 'Ganho Capital', 'Perda Capital']

### Chama a função que retorna os valores de X e y

In [21]:
X, y = prep.retorna_data_x_y(lista_colunas_dummy, lista_colunas_scale, renda3[lista_colunas_aux])

# Balanceamento da base

###  Chama função para balanceamento da base de dados

In [22]:
X_balanc, y_balanc = prep.retorna_data_x_y_balanceada(X, y)

# Criação dos modelos

### Algoritmo XGB

In [23]:
# Separa base de treino e teste
X_trainXGB, X_testXGB, y_trainXGB, y_testXGB = train_test_split(X_balanc, y_balanc, test_size=0.2, random_state=1)

# Cria uma instancia do algoritmo XGB
modeloXGB = XGBClassifier(objective='binary:logistic', 
                          learning_rate=0.1, 
                          max_depth=9, 
                          n_estimators=180,
                          nthread=4, 
                          seed=42)
# Treina o modelo
modeloXGB.fit(X_trainXGB, y_trainXGB, eval_metric='rmse')

# Faz a predição
y_predXGB = modeloXGB.predict(X_testXGB)

### Algoritmo Random Forest

In [24]:
# Separa base de treino e teste
X_trainRF, X_testRF, y_trainRF, y_testRF = train_test_split(X_balanc, y_balanc, test_size=0.2, random_state=1)

# Cria uma instancia do algoritmo Random Forest
modeloRF = RandomForestClassifier(bootstrap=True, 
                                  max_depth=70, 
                                  max_features=2, 
                                  min_samples_leaf=3, 
                                  min_samples_split=4, 
                                  n_estimators=700,
                                  random_state=1)                                  
# Treina o modelo
modeloRF.fit(X_trainRF, y_trainRF.values.ravel())

# Faz a predição
y_predRF = modeloRF.predict(X_testRF)

### Algoritmo Decision Tree

In [25]:
# Separa base de treino e teste
X_trainDT, X_testDT, y_trainDT, y_testDT = train_test_split(X_balanc, y_balanc, test_size=0.2, random_state=1)

# Cria uma instancia do algoritmo Decison Tree
modeloDT = DecisionTreeClassifier(criterion='entropy', 
                                  max_depth=14,
                                  random_state=1)
                                                                    
# Treina o modelo
modeloDT.fit(X_trainDT, y_trainDT)

# Faz a predição
y_predDT = modeloDT.predict(X_testDT)

### Algoritmo SVM

In [26]:
# Separa base de treino e teste
X_trainSVM, X_testSVM, y_trainSVM, y_testSVM = train_test_split(X_balanc, y_balanc, test_size=0.2, random_state=1)

# Cria uma instancia do algoritmo Decison Tree
modeloSVM = SVC(C=100, 
                gamma=1,
                kernel='poly',
                random_state=1)
                                                                    
# Treina o modelo
modeloSVM.fit(X_trainSVM, y_trainSVM)

# Faz a predição
y_predSVM = modeloSVM.predict(X_testSVM)

# Validação do Modelo

* Será usada para validação do modelo a validação cruzada da biblioteca scikitlearn

In [34]:
def validacao_cruzada_modelo(modelo, x_train, y_train):
    val_scores = cross_val_score(modelo, x_train, y_train, cv=5)
    print('Acurácia nos k-folds:', val_scores)
    print('Média: {:.2} | Desvio: {:.2}'.format(np.mean(val_scores), np.std(val_scores)))

### Algoritmo XGB

In [35]:
validacao_cruzada_modelo(modeloXGB, X_trainXGB, y_trainXGB)

[15:09:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:09:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:09:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:09:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

### Algoritmo Random Forest

In [36]:
validacao_cruzada_modelo(modeloRF, X_trainRF, y_trainRF)

c:\users\marcos.fusato\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\users\marcos.fusato\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\users\marcos.fusato\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\users\marcos.fus

Acurácia nos k-folds: [0.99150066 0.99176627 0.99070385 0.98831341 0.99150066]
Média: 0.99 | Desvio: 0.0013


### Algoritmo Decision Tree    

In [37]:
validacao_cruzada_modelo(modeloDT, X_trainDT, y_trainDT)

Acurácia nos k-folds: [0.99176627 0.99096946 0.99468792 0.99096946 0.9938911 ]
Média: 0.99 | Desvio: 0.0015


### Algoritmo SVM

In [38]:
validacao_cruzada_modelo(modeloSVM, X_trainSVM, y_trainSVM)

Acurácia nos k-folds: [0.95962815 0.96812749 0.9564409  0.96440903 0.95776892]
Média: 0.96 | Desvio: 0.0044


# Avaliação do Modelo

### Acuracia

In [39]:
print("Acuracia para o algoritmo XGB é: %.2f%%" % (accuracy_score(y_testXGB, y_predXGB) * 100))
print("Acuracia para o algoritmo Random Forest é: %.2f%%" % (accuracy_score(y_testRF, y_predRF) * 100))
print("Acuracia para o algoritmo Decison Tree é: %.2f%%" % (accuracy_score(y_testDT, y_predDT) * 100))
print("Acuracia para o algoritmo SVM é: %.2f%%" % (accuracy_score(y_testSVM, y_predSVM) * 100))

Acuracia para o algoritmo XGB é: 99.34%
Acuracia para o algoritmo Random Forest é: 99.17%
Acuracia para o algoritmo Decison Tree é: 99.30%
Acuracia para o algoritmo SVM é: 96.49%


### Matriz de Confusão

In [40]:
print('Matriz de Confusão Algoritmo XGB: \n', confusion_matrix(y_testXGB, y_predXGB), '\n')
print('Matriz de Confusão Algoritmo Random Forest: \n', confusion_matrix(y_testRF, y_predRF), '\n')
print('Matriz de Confusão Algoritmo Decison Tree: \n', confusion_matrix(y_testDT, y_predDT), '\n')
print('Matriz de Confusão Algoritmo SVM: \n', confusion_matrix(y_testSVM, y_predSVM), '\n')

Matriz de Confusão Algoritmo XGB: 
 [[2635    9]
 [  22 2041]] 

Matriz de Confusão Algoritmo Random Forest: 
 [[2634   10]
 [  29 2034]] 

Matriz de Confusão Algoritmo Decison Tree: 
 [[2636    8]
 [  25 2038]] 

Matriz de Confusão Algoritmo SVM: 
 [[2543  101]
 [  64 1999]] 



### Precisão

In [41]:
print("Precisão para o algoritmo XGB é: %.2f%%" % (precision_score(y_testXGB, y_predXGB) * 100))
print("Precisão para o algoritmo Random Forest é: %.2f%%" % (precision_score(y_testRF, y_predRF) * 100))
print("Precisão para o algoritmo Decison Tree é: %.2f%%" % (precision_score(y_testDT, y_predDT) * 100))
print("Precisão para o algoritmo SVM é: %.2f%%" % (precision_score(y_testSVM, y_predSVM) * 100))

Precisão para o algoritmo XGB é: 99.56%
Precisão para o algoritmo Random Forest é: 99.51%
Precisão para o algoritmo Decison Tree é: 99.61%
Precisão para o algoritmo SVM é: 95.19%


### Recall

In [42]:
print("Recall para o algoritmo XGB é: %.2f%%" % (recall_score(y_testXGB, y_predXGB) * 100))
print("Recall para o algoritmo Random Forest é: %.2f%%" % (recall_score(y_testRF, y_predRF) * 100))
print("Recall para o algoritmo Decison Tree é: %.2f%%" % (recall_score(y_testDT, y_predDT) * 100))
print("Recall para o algoritmo SVM é: %.2f%%" % (recall_score(y_testSVM, y_predSVM) * 100))

Recall para o algoritmo XGB é: 98.93%
Recall para o algoritmo Random Forest é: 98.59%
Recall para o algoritmo Decison Tree é: 98.79%
Recall para o algoritmo SVM é: 96.90%


### F1 Score

In [43]:
print("F1 Score para o algoritmo XGB é: %.2f%%" % (f1_score(y_testXGB, y_predXGB) * 100))
print("F1 Score para o algoritmo Random Forest é: %.2f%%" % (f1_score(y_testRF, y_predRF) * 100))
print("F1 Score para o algoritmo Decison Tree é: %.2f%%" % (f1_score(y_testDT, y_predDT) * 100))
print("F1 Score para o algoritmo SVM é: %.2f%%" % (f1_score(y_testSVM, y_predSVM) * 100))

F1 Score para o algoritmo XGB é: 99.25%
F1 Score para o algoritmo Random Forest é: 99.05%
F1 Score para o algoritmo Decison Tree é: 99.20%
F1 Score para o algoritmo SVM é: 96.04%


### Curva AUC-ROC

In [44]:
fprXGB, tprXGB, thresholdXGB = roc_curve(y_testXGB, y_predXGB)
roc_aucXGB = auc(fprXGB, tprXGB)
print("Curva AUC-ROC para o algoritmo XGB é: %.2f%%" % (roc_aucXGB * 100))

fprRF, tprRF, thresholdRF = roc_curve(y_testRF, y_predRF)
roc_aucRF = auc(fprRF, tprRF)
print("Curva AUC-ROC para o algoritmo Random Forest é: %.2f%%" % (roc_aucRF * 100))

fprDT, tprDT, thresholdDT = roc_curve(y_testDT, y_predDT)
roc_aucDT = auc(fprDT, tprDT)
print("Curva AUC-ROC para o algoritmo Decision Tree é: %.2f%%" % (roc_aucDT * 100))

fprSVM, tprSVM, thresholdSVM = roc_curve(y_testSVM, y_predSVM)
roc_aucSVM = auc(fprSVM, tprSVM)
print("Curva AUC-ROC para o algoritmo SVM é: %.2f%%" % (roc_aucSVM * 100))

Curva AUC-ROC para o algoritmo XGB é: 99.30%
Curva AUC-ROC para o algoritmo Random Forest é: 99.11%
Curva AUC-ROC para o algoritmo Decision Tree é: 99.24%
Curva AUC-ROC para o algoritmo SVM é: 96.54%


In [ ]:
#metricas
#https://medium.com/analytics-vidhya/evaluation-metrics-for-classification-problems-with-implementation-in-python-a20193b4f2c3